In [1]:
## -----------IMPORT LIBRARIES
import pandas as pd #for dealing with csv import
import os # for joining paths and filenames sensibly
import numpy as np #for the population std
import glob # for finding csv data Çfiles
import platform # paths use different dividers on linux vs windows, so we need to test for this
import csv

In [3]:
path_folder = 'C:\\Users\\David\\Desktop\\OpenBrains\\biased_brains'

exclude_ = ['coles', 'data', 'eyetracker_recordings', 'guinardo_all', 'juncar_all', 'other', 'implicit_bias_imerys' ]

folders = os.listdir(path_folder)

os.listdir(path_folder)

['coles',
 'data',
 'eyetracker_recordings',
 'guinardo_all',
 'guinardo_all2',
 'guinardo_bt',
 'implicit_bias_imerys',
 'juncar_1',
 'juncar_2',
 'juncar_all',
 'maristes',
 'marti_pous',
 'Moli',
 'Moli_bt',
 'other',
 'PiaBalaguer',
 'Pias_sc',
 'Pias_sc_3D',
 'Pias_sc_bt']

In [6]:
folder_ = set(folders) - set(exclude_)
folders_correct = list(folder_)
folders_correct

['PiaBalaguer',
 'juncar_1',
 'maristes',
 'Pias_sc',
 'Pias_sc_bt',
 'marti_pous',
 'Moli_bt',
 'guinardo_all2',
 'Pias_sc_3D',
 'guinardo_bt',
 'Moli',
 'juncar_2']

In [10]:
all_files = []

for idx,fold in enumerate(folders_correct):
    files_ = os.path.join(path_folder, folders_correct[idx])
    search_string=os.path.join(files_,'*.csv') 
    files = glob.glob(search_string)
    all_files.append(files)

    
IAT_files = [item for sublist in all_files for item in sublist]


In [12]:
len(IAT_files)

467